In [ ]:
# To destroy.
import digitalocean
import os
import datetime
from tqdm import tqdm
project_to_destroy = "W"
to_destroy = []
manager = digitalocean.Manager( token = os.getenv( 'MARIUS_DOTOKEN' ) ) 
my_droplets = manager.get_all_droplets()
for droplet in my_droplets:
    if len(droplet.tags) > 0 and droplet.tags[0] == project_to_destroy:
        to_destroy.append(droplet)
pbar = tqdm(range(0, len(to_destroy)), "Destroying Droplets")
for droplet in to_destroy:
    pbar.set_description("Destroying %s" % droplet.name)
    droplet.destroy()
    pbar.update(1)


In [ ]:
# To create.
import digitalocean
import os
from tqdm import tqdm
manager = digitalocean.Manager( token = os.getenv( 'MARIUS_DOTOKEN' ) ) 
project = "R"
slug = 's-8vcpu-16gb'
region = 'nyc3'
#n_to_create = 200
keys = manager.get_all_sshkeys()
projects = manager.get_all_projects()
pbar = tqdm(range(300, 500), "Creating Droplets")
for i in pbar:
    name = "{}{}".format(project, i)
    pbar.set_description("Creating %s" % name)
    droplet = digitalocean.Droplet(token = os.getenv( 'MARIUS_DOTOKEN' ), name=name, region=region, image='ubuntu-20-04-x64', size_slug=slug, backups=True, tags = [project], ssh_keys = keys)
    droplet.create()
    for p in projects:
        if p.name == project:
            p.assign_resource(["do:droplet:{}".format(droplet.id)])
            break

In [ ]:
# Create wallets
import bittensor
from tqdm import tqdm
project = "R"
manager = digitalocean.Manager( token = os.getenv( 'MARIUS_DOTOKEN' ) ) 
my_droplets = manager.get_all_droplets()
wallet = bittensor.wallet(name = project)
if not wallet.coldkey_file.exists_on_device():
    wallet.create_new_coldkey()
for droplet in my_droplets:
    wallet = bittensor.wallet(name = project, hotkey=droplet.name )
    if not wallet.hotkey_file.exists_on_device():
        wallet.create_new_hotkey()


In [ ]:
# Append droplets to config
from hud import HUD
project = "R"
neurons = HUD.load_from_digital_ocean( tag = project )
HUD.write_to_config( neurons )
neurons = HUD.load_from_config( project = project )
print( neurons )
